In [1]:
import copy
import numpy as np
import os
import verdict

In [2]:
import matplotlib
import matplotlib.pyplot as plt

In [3]:
import galaxy_dive.read_data.metafile as read_metafile
import galaxy_dive.plot_data.plotting as plotting
import galaxy_dive.utils.utilities as utilities
import galaxy_dive.utils.executable_helpers as exec_helpers

In [4]:
import linefinder.utils.file_management as file_management
import linefinder.analyze_data.worldline_set as worldline_set
import linefinder.analyze_data.worldlines as worldlines
import linefinder.utils.presentation_constants as p_constants
import linefinder.config as l_config

In [5]:
import analysis_config

# Load Data

In [6]:
do_calculation = True

In [7]:
snum, galdef = exec_helpers.choose_config_or_commandline(
    [ analysis_config.SNUM, analysis_config.GALDEF ]
)
print( 'Using snum {}, galdef {}'.format( snum, galdef ) )

Using snum 465, galdef 


In [8]:
presentation_figure = False

In [9]:
mass_key = 'M'

In [10]:
save_file_tags = {
    'M' : 'mass',
    'metal_mass' : 'metalmass',
    'enriched_metal_mass' : 'enrichedmetalmass',
}

In [11]:
axes_labels = {
    'M' : 'Mass',
    'metal_mass' : 'Metal Mass',
    'enriched_metal_mass' : 'Metal Mass',
}

In [12]:
file_manager = file_management.FileManager( project='CGM_fate' )

In [13]:
ind = 600 - snum

In [14]:
tag_tail = '_CGM_snum{}'.format( snum )

In [15]:
defaults, variations = file_manager.get_linefinder_analysis_defaults_and_variations(
    tag_tail,
    sim_names = analysis_config.SIM_NAMES,
    galdef = galdef,
)
defaults, variations

({'data_dir': '/scratch/03057/zhafen/linefinder_data/core/m12i_res7100/data',
  'tag': 'm12i_CGM_snum465',
  'halo_data_dir': '/scratch/03057/zhafen/core/m12i_res7100/halo',
  'ahf_index': 600,
  'main_halo_id': 0},
 {'m10q': {'data_dir': '/scratch/03057/zhafen/linefinder_data/core/m10q_res250/data',
   'tag': 'm10q_CGM_snum465',
   'halo_data_dir': '/scratch/03057/zhafen/core/m10q_res250/halo',
   'ahf_index': 600,
   'main_halo_id': 0},
  'm10v': {'data_dir': '/scratch/03057/zhafen/linefinder_data/core/m10v_res250/data',
   'tag': 'm10v_CGM_snum465',
   'halo_data_dir': '/scratch/03057/zhafen/core/m10v_res250/halo',
   'ahf_index': 600,
   'main_halo_id': 2},
  'm10y': {'data_dir': '/scratch/03057/zhafen/linefinder_data/core/m10y_res250/data',
   'tag': 'm10y_CGM_snum465',
   'halo_data_dir': '/scratch/03057/zhafen/core/m10y_res250/halo',
   'ahf_index': 600,
   'main_halo_id': 0},
  'm10z': {'data_dir': '/scratch/03057/zhafen/linefinder_data/core/m10z_res250/data',
   'tag': 'm10z_C

In [16]:
w_set = worldline_set.WorldlineSet( defaults, variations )

In [17]:
default_sim_name = list( w_set.keys() )[0]

In [18]:
w = w_set[default_sim_name]

In [19]:
classifications = copy.deepcopy( p_constants.CLASSIFICATIONS_CGM_FATE )
classifications.append( 'is_in_CGM_not_sat' )

In [20]:
metafile_reader = read_metafile.MetafileReader(
    file_manager.get_metafile_dir( default_sim_name )
)

# Analyze Data

### Get masses out

In [21]:
w_set.data_object.data_masker.clear_masks()

Dict, {
'm10q' : None,
'm10v' : None,
'm10y' : None,
'm10z' : None,
'm11q' : None,
'm11v' : None,
'm11a' : None,
'm11b' : None,
'm11c' : None,
'm12i' : None,
'm12f' : None,
'm12m' : None,
'm11d_md' : None,
'm11e_md' : None,
'm11h_md' : None,
'm11i_md' : None,
'm12b_md' : None,
'm12c_md' : None,
'm12z_md' : None,
'm12r_md' : None,
'm12w_md' : None,
}

#### Mass Fractions

In [22]:
import h5py

In [23]:
f = h5py.File( '/scratch/03057/zhafen/linefinder_data/core/m10q_res250/data/classifications_m10q_CGM_snum465.hdf5', 'r' )

In [24]:
list( f.keys() )

['CGM_fate_classifications',
 'is_CGM_IGM_accretion',
 'is_CGM_satellite_ISM',
 'is_CGM_satellite_wind',
 'is_CGM_wind',
 'is_hitherto_EP',
 'is_hitherto_NEP',
 'is_in_CGM',
 'is_in_galaxy_halo_interface',
 'is_mass_transfer',
 'is_merger',
 'is_preprocessed',
 'is_pristine',
 'is_unaccreted',
 'is_unaccreted_EP',
 'is_unaccreted_NEP',
 'is_wind',
 'parameters']

In [26]:
if do_calculation:
    
    CGM_mass_fractions = {}
    CGM_mass_fractions['and'] = {}

    # See what different fates consist of
    for classification in classifications:
        
        assert False, "Need to optimize, I'm deleting eeeverything...."
        
        print( classification )

        # Sometimes we want to look at how things are relatively
        for sim_name in w_set.keys():
                        
            for joint, normalization_category in zip( [ False, True], [ classification, 'is_in_CGM_not_sat' ] ):

                if classification == 'is_in_CGM_not_sat':
                    inner_classifications = classifications
                else:
                    inner_classifications = p_constants.CLASSIFICATIONS_CGM_ORIGIN

                CGM_mass_fractions_sim = verdict.Dict( {} )
                
                normalization_mass = w_set[sim_name].data_object.get_selected_quantity(
                    selection_routine = normalization_category,
                    ptype = 'gas',
                    sl = (slice(None),ind),
                    selected_quantity_data_key = mass_key,
                )
                
                # When there's no mass in the outer category
                if np.isclose( normalization_mass, 0. ):
                    CGM_mass_fractions_sim = verdict.Dict( {} )
                    for inner_class in inner_classifications:
                        CGM_mass_fractions_sim[inner_class] = 0.0
                # Standard case
                else:
                    for i_c in inner_classifications:
                        CGM_mass_fractions_sim[i_c] = w_set[sim_name].data_object.get_selected_quantity(
                            selection_routine = [ classification, i_c ],
                            ptype = 'gas',
                            sl = (slice(None),ind),
                            selected_quantity_data_key = mass_key,
                        )
                    
                    CGM_mass_fractions_sim /= normalization_mass
                
                try:
                    CGM_mass_fractions_cat[sim_name] = CGM_mass_fractions_sim
                except NameError:
                    CGM_mass_fractions_cat = verdict.Dict( {} )
                    CGM_mass_fractions_cat[sim_name] = CGM_mass_fractions_sim
                
            w_set[sim_name].data_object.clear_data()

            CGM_mass_fractions_cat = CGM_mass_fractions_cat.transpose()

            if joint:
                CGM_mass_fractions['and'][classification] = CGM_mass_fractions_cat
            else:
                CGM_mass_fractions[classification] = CGM_mass_fractions_cat
        
    # See where different origins go
#     for classification in p_constants.CLASSIFICATIONS_CGM_ORIGIN:
        
#         print( classification )

#         CGM_mass_fractions_cat = verdict.Dict( {} )
#         for sim_name in w_set.keys():
            
#             inner_classifications = p_constants.CLASSIFICATIONS_CGM_FATE
            
#             try:
#                 CGM_mass_fractions_cat[sim_name] = w_set[sim_name].data_object.get_categories_selected_quantity_fraction(
#                     normalization_category = classification,
#                     selection_routine = classification,
#                     ptype = 'gas',
#                     classification_list = inner_classifications,
#                     sl = (slice(None),ind),
#                     selected_quantity_data_key = mass_key,
#                 )
#             except ZeroDivisionError:
#                 mass_fractions_sim = verdict.Dict( {} )
#                 for inner_class in inner_classifications:
#                     mass_fractions_sim[inner_class] = 0.0
#                 CGM_mass_fractions_cat[sim_name] = mass_fractions_sim

#         CGM_mass_fractions_cat = CGM_mass_fractions_cat.transpose()

#         CGM_mass_fractions[classification] = CGM_mass_fractions_cat

    CGM_mass_fractions = verdict.Dict( CGM_mass_fractions )

AssertionError: Need to optimize, I'm deleting eeeverything....

#### Halo Masses

In [ ]:
if do_calculation:    
    w_set.data_object.retrieve_halo_data()
    halo_masses = w_set.m_vir.inner_item( snum )

#### Save and load results

In [ ]:
savefile = os.path.join(
    file_manager.project_parameters['output_data_dir'],
    'cgm_fates_{}_frac_snum{}.hdf5'.format( save_file_tags[mass_key], snum ),
)

In [ ]:
if do_calculation:

    # Format results to save
    results_to_save = copy.deepcopy( CGM_mass_fractions )
    results_to_save['m_vir'] = halo_masses

    results_to_save.to_hdf5(
        savefile, 
        condensed = True, 
        attributes = { 'redshift': w.redshift.values[ind] },
    )
    
    redshift = w.redshift.values[ind]

In [ ]:
if not do_calculation:
    results_to_load, attributes = verdict.Dict.from_hdf5( savefile, unpack=True )
    
    halo_masses = results_to_load['m_vir']
    CGM_mass_fractions = copy.deepcopy( results_to_load )
    del CGM_mass_fractions['m_vir']
    
    redshift = attributes['redshift']

# Plot Data

## Setup

In [ ]:
mass_range = [ halo_masses.array().min()/1.5, halo_masses.array().max()*1.5 ]
mass_range

In [ ]:
m_vir_md_split = halo_masses.split_by_key_slice( slice(4,10), '_md' )
m_vir_fiducials = m_vir_md_split[False]
m_vir_mds = m_vir_md_split[True]

In [ ]:
def save_plot_stage( fig, base_save_file, index ):

    save_file = '{}.{}.pdf'.format( base_save_file, index )
    
    plotting.save_fig(
        out_dir = file_manager.get_project_presentation_dir(),
        save_file = save_file,
        fig = fig,
    )
    
    return index + 1

In [ ]:
CGM_mass_fractions

## Mass Fraction Plot

In [ ]:
fig = plt.figure( figsize=(10,8), facecolor='w' )
ax = plt.gca()

# Some plot settings
y_min = 1e-2
y_max = 1
alpha = 1.0

# Plot data points
color_objects = []
labels = []
j = 0
for k, classification in enumerate( p_constants.CLASSIFICATIONS_CGM_FATE ):
    
    if presentation_figure:
                
        j = save_plot_stage( fig, 'CGM_{}_frac_vs_Mh{}'.format( save_file_tags[mass_key], tag_tail ), j )
    
    if classification == 'is_in_CGM_not_sat':
        continue
        
    item = CGM_mass_fractions['is_in_CGM_not_sat'][classification]
    
    md_split = item.split_by_key_slice( slice(4,10), '_md' )
    fiducials = md_split[False]
    mds = md_split[True]
    
    edgecolor = np.array( matplotlib.colors.colorConverter.to_rgba(
        l_config.COLORSCHEME[classification]
    ) )
    edgecolor[-1] = alpha
        
    # Default points
    ax.scatter(
        m_vir_fiducials.array(),
        fiducials.array(),
        s = 130,
        color = None,
        zorder = 100 - k,
        linewidth = 0,
        facecolors = l_config.COLORSCHEME[classification],
        alpha = alpha,
    )
    
    if mass_key != 'M':
        facecolors = 'none'
        color = edgecolor
    else:
        facecolors = l_config.COLORSCHEME[classification]
        color = 'none'
        
    # Turbulent metal diffusion
    ax.scatter(
        m_vir_mds.array(),
        mds.array(),
        s = 120,
        color = color,
        marker = 'o',
        zorder = 100 - k,
        facecolors = facecolors,
        linewidth = 3.5,
        alpha = alpha,
    )
    
    # Box plot
    if mass_key != 'M':
        x_data_list = [ m_vir_fiducials, m_vir_mds, ]
        y_data_list = [ fiducials, mds, ]
        linestyles = [ '-', '--', ]
    else:
        x_data_list = [ halo_masses, ]
        y_data_list = [ item, ]
        linestyles = [ '-', ]
    for x_data, y_data, linestyle in zip( x_data_list, y_data_list, linestyles ):
        x_datas = x_data.split_by_dict( l_config.MASS_BINS, return_list=True )
        y_datas = y_data.split_by_dict( l_config.MASS_BINS, return_list=True )
        plotting.box_plot(
            x_datas,
            y_datas,
            ax = ax,
            color = l_config.COLORSCHEME[classification],
            box_zorder = 50 - k,
            blank_zorder = 30 - k,
            line_zorder = 10 - k,
    #         linewidth = 7 - k,
            linewidth = 5,
            linestyle = linestyle,
            y_floor = 1e-5,
            plot_boxes = False,
            line_x_min = x_data.array().min(),
            line_x_max = x_data.array().max(),
            y_mean_statistic = np.median,
        )
    
    if classification is not None:
        # Make virtual artists to allow a legend to appear
        color_object = matplotlib.patches.Rectangle(                         
            (0, 0),                                                          
            1,                                                               
            1,                                                               
            fc = l_config.COLORSCHEME[classification],                                 
            ec = l_config.COLORSCHEME[classification],                                 
            alpha = p_constants.CLASSIFICATION_ALPHA,                        
        )
        color_objects.append( color_object )                                 
        labels.append( p_constants.CLASSIFICATION_LABELS[classification] )

    # Add a redshift label
    if k == 0:
        redshift_label = r'$z=' + '{:.02g}'.format( redshift ) + '$'
        ax.annotate( s=redshift_label, xy=(0.05,1.0125), xycoords='axes fraction', fontsize=22 )

    if snum == 172:
        ax.set_xlabel( r'$M_{\rm h}$ ($M_{\odot}$)', fontsize=24 )
    ax.set_ylabel( r'CGM {} Fraction'.format( axes_labels[mass_key] ), fontsize=24 )

    ax.set_xlim( 2e9, 2e12 )
    ax.set_ylim( y_min, y_max )

    ax.set_xscale( 'log' )
    ax.set_yscale( 'log' )
    
    # Add ticks to the righ
    ax.yaxis.set_ticks_position( 'both' )

    if snum == 172:
        legend = ax.legend(
            color_objects,
            labels,
            prop={'size': 17.5},
            ncol=1,
            loc='lower right',
            fontsize=24,
            framealpha = 0.9,
        )
        legend.set_zorder( 120 )

save_file = 'CGM_{}_frac_vs_Mh{}.pdf'.format( save_file_tags[mass_key], tag_tail )

plotting.save_fig(
    out_dir = file_manager.get_project_figure_dir(),
    save_file = save_file,
    fig = fig,
)

# Connection Between Origin and Fate

In [ ]:
outer_classifications = p_constants.CLASSIFICATIONS_CGM_FATE
inner_classifications = p_constants.CLASSIFICATIONS_CGM_ORIGIN

n_rows = len( outer_classifications )

fig = plt.figure( figsize=(10,n_rows*4), facecolor='w' )
main_ax = plt.gca()

gs = matplotlib.gridspec.GridSpec( n_rows, 1, )
gs.update( hspace=0.00001 )

for i, outer_classification in enumerate( outer_classifications ):
    
    ax = plt.subplot( gs[i,0] )
    
    for j, classification in enumerate( inner_classifications ):
        
        item = CGM_mass_fractions[outer_classification][classification]

        md_split = item.split_by_key_slice( slice(4,10), '_md' )
        fiducials = md_split[False]
        mds = md_split[True]

        edgecolor = np.array( matplotlib.colors.colorConverter.to_rgba(
            l_config.COLORSCHEME[classification]
        ) )
        edgecolor[-1] = alpha

        # Default points
        ax.scatter(
            m_vir_fiducials.array(),
            fiducials.array(),
            s = 130,
            color = None,
            zorder = 100 - k,
            linewidth = 0,
            facecolors = l_config.COLORSCHEME[classification],
            alpha = alpha,
        )

        if mass_key != 'M':
            facecolors = 'none'
            color = edgecolor
        else:
            facecolors = l_config.COLORSCHEME[classification]
            color = 'none'

        # Turbulent metal diffusion
        ax.scatter(
            m_vir_mds.array(),
            mds.array(),
            s = 120,
            color = color,
            marker = 'o',
            zorder = 100 - k,
            facecolors = facecolors,
            linewidth = 3.5,
            alpha = alpha,
        )

        # Box plot
        if mass_key != 'M':
            x_data_list = [ m_vir_fiducials, m_vir_mds, ]
            y_data_list = [ fiducials, mds, ]
            linestyles = [ '-', '--', ]
        else:
            x_data_list = [ halo_masses, ]
            y_data_list = [ item, ]
            linestyles = [ '-', ]
        for x_data, y_data, linestyle in zip( x_data_list, y_data_list, linestyles ):
            x_datas = x_data.split_by_dict( l_config.MASS_BINS, return_list=True )
            y_datas = y_data.split_by_dict( l_config.MASS_BINS, return_list=True )
            plotting.box_plot(
                x_datas,
                y_datas,
                ax = ax,
                color = l_config.COLORSCHEME[classification],
                box_zorder = 50 - k,
                blank_zorder = 30 - k,
                line_zorder = 10 - k,
        #         linewidth = 7 - k,
                linewidth = 5,
                linestyle = linestyle,
                y_floor = 1e-5,
                plot_boxes = False,
                line_x_min = x_data.array().min(),
                line_x_max = x_data.array().max(),
                y_mean_statistic = np.median,
            )

        if classification is not None:
            # Make virtual artists to allow a legend to appear
            color_object = matplotlib.patches.Rectangle(                         
                (0, 0),                                                          
                1,                                                               
                1,                                                               
                fc = l_config.COLORSCHEME[classification],                                 
                ec = l_config.COLORSCHEME[classification],                                 
                alpha = p_constants.CLASSIFICATION_ALPHA,                        
            )
            color_objects.append( color_object )                                 
            labels.append( p_constants.CLASSIFICATION_LABELS[classification] )

        # Add a redshift label
        if k == 0:
            redshift_label = r'$z=' + '{:.02g}'.format( redshift ) + '$'
            ax.annotate( s=redshift_label, xy=(0.05,1.0125), xycoords='axes fraction', fontsize=22 )

        if snum == 172:
            ax.set_xlabel( r'$M_{\rm h}$ ($M_{\odot}$)', fontsize=24 )
        ax.set_ylabel( p_constants.CLASSIFICATION_LABELS[outer_classification], fontsize=24 )

        ax.set_xlim( 2e9, 2e12 )
        ax.set_ylim( y_min, y_max )

        ax.set_xscale( 'log' )
        ax.set_yscale( 'log' )

        # Add ticks to the right
        ax.yaxis.set_ticks_position( 'both' )

        if snum == 172:
            legend = ax.legend(
                color_objects,
                labels,
                prop={'size': 17.5},
                ncol=1,
                loc='lower right',
                fontsize=24,
                framealpha = 0.9,
            )
            legend.set_zorder( 120 )

In [ ]:
inner_classifications = p_constants.CLASSIFICATIONS_CGM_FATE
outer_classifications = p_constants.CLASSIFICATIONS_CGM_ORIGIN

n_rows = len( outer_classifications )

fig = plt.figure( figsize=(10,n_rows*4), facecolor='w' )
main_ax = plt.gca()

gs = matplotlib.gridspec.GridSpec( n_rows, 1, )
gs.update( hspace=0.00001 )

color_objects = []
for i, outer_classification in enumerate( outer_classifications ):
    
    ax = plt.subplot( gs[i,0] )
    
    for j, classification in enumerate( inner_classifications ):
        
        item = CGM_mass_fractions['and'][classification][outer_classification]

        md_split = item.split_by_key_slice( slice(4,10), '_md' )
        fiducials = md_split[False]
        mds = md_split[True]

        edgecolor = np.array( matplotlib.colors.colorConverter.to_rgba(
            l_config.COLORSCHEME[classification]
        ) )
        edgecolor[-1] = alpha

        # Default points
        ax.scatter(
            m_vir_fiducials.array(),
            fiducials.array(),
            s = 130,
            color = None,
            zorder = 100 - k,
            linewidth = 0,
            facecolors = l_config.COLORSCHEME[classification],
            alpha = alpha,
        )

        if mass_key != 'M':
            facecolors = 'none'
            color = edgecolor
        else:
            facecolors = l_config.COLORSCHEME[classification]
            color = 'none'

        # Turbulent metal diffusion
        ax.scatter(
            m_vir_mds.array(),
            mds.array(),
            s = 120,
            color = color,
            marker = 'o',
            zorder = 100 - k,
            facecolors = facecolors,
            linewidth = 3.5,
            alpha = alpha,
        )

        # Box plot
        if mass_key != 'M':
            x_data_list = [ m_vir_fiducials, m_vir_mds, ]
            y_data_list = [ fiducials, mds, ]
            linestyles = [ '-', '--', ]
        else:
            x_data_list = [ halo_masses, ]
            y_data_list = [ item, ]
            linestyles = [ '-', ]
        for x_data, y_data, linestyle in zip( x_data_list, y_data_list, linestyles ):
            x_datas = x_data.split_by_dict( l_config.MASS_BINS, return_list=True )
            y_datas = y_data.split_by_dict( l_config.MASS_BINS, return_list=True )
            plotting.box_plot(
                x_datas,
                y_datas,
                ax = ax,
                color = l_config.COLORSCHEME[classification],
                box_zorder = 50 - k,
                blank_zorder = 30 - k,
                line_zorder = 10 - k,
        #         linewidth = 7 - k,
                linewidth = 5,
                linestyle = linestyle,
                y_floor = 1e-5,
                plot_boxes = False,
                line_x_min = x_data.array().min(),
                line_x_max = x_data.array().max(),
                y_mean_statistic = np.median,
            )

        if classification is not None and classification in inner_classifications and ax.is_first_row():
            # Make virtual artists to allow a legend to appear
            color_object = matplotlib.patches.Rectangle(                         
                (0, 0),                                                          
                1,                                                               
                1,                                                               
                fc = l_config.COLORSCHEME[classification],                                 
                ec = l_config.COLORSCHEME[classification],                                 
                alpha = p_constants.CLASSIFICATION_ALPHA,                        
            )
            color_objects.append( color_object )                                 
            labels.append( p_constants.CLASSIFICATION_LABELS[classification] )

    # Add a redshift label
    if ax.is_first_row():
        redshift_label = r'$z=' + '{:.02g}'.format( redshift ) + '$'
        ax.annotate( s=redshift_label, xy=(0.05,1.0125), xycoords='axes fraction', fontsize=22 )

    # Classification Labels
    ax.annotate(
        s = 'Fate + ' + p_constants.CLASSIFICATION_LABELS[outer_classification],
        xy = ( 0.05, 0.05 ),
        xycoords = 'axes fraction',
        fontsize = 21,
        ha = 'left',
        va = 'bottom',
        zorder = 200,
    )
    
    # Axes Labels
    if ax.is_last_row():
        ax.set_xlabel( r'$M_{\rm h}$ ($M_{\odot}$)', fontsize=24 )
    ax.set_ylabel( r'$f(M_{\rm CGM})$', fontsize=24 )

    ax.set_xlim( 2e9, 2e12 )
    ax.set_ylim( y_min, y_max )

    ax.set_xscale( 'log' )
    ax.set_yscale( 'log' )

    # Avoid overlapping ticks
    ax.get_yticklabels()[1].set_verticalalignment( 'bottom' )
    ax.get_yticklabels()[-2].set_verticalalignment( 'top' )
    
    # Add ticks to the right
    ax.yaxis.set_ticks_position( 'both' )

    if snum == 172 and ax.is_last_row():
        legend = ax.legend(
            color_objects,
            labels,
            prop={'size': 17.5},
            ncol=1,
            loc='upper left',
            fontsize=24,
            framealpha = 0.9,
        )
        legend.set_zorder( 120 )
        
save_file = 'originfate_{}_frac_vs_Mh{}.pdf'.format( save_file_tags[mass_key], tag_tail )

plotting.save_fig(
    out_dir = file_manager.get_project_figure_dir(),
    save_file = save_file,
    fig = fig,
)

# Check if the above is correct

Below checks that everything sums to 1.

In [ ]:
try:
    del total
except NameError:
    pass
total = verdict.Dict( {} )
for classification in p_constants.CLASSIFICATIONS_CGM_FATE:
    for inner_classification in p_constants.CLASSIFICATIONS_CGM_ORIGIN:
        try:
            total[classification] += CGM_mass_fractions['and'][classification][inner_classification]
        except KeyError:
            total[classification] = CGM_mass_fractions['and'][classification][inner_classification]
total.sum_contents()

In [ ]:
min_sim, min_val = total.sum_contents().keymin()
min_sim, min_val

In [ ]:
assert min_val > 0.95, "Missing categories!"

In [ ]:
max_sim, max_val = total.sum_contents().keymax()
max_sim, max_val

In [ ]:
assert max_val < 1.05, "Overlapping categories!"